In [5]:
import torch
from train_cnn import *

from loaddata import *
import pandas as pd

DATA_DIR = '../data'
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
BATCHSIZE = 100

PATH_MNISTC = '../data/MNIST_C/'
CORRUPTION_TYPES = ['identity', 
         'shot_noise', 'impulse_noise','glass_blur','motion_blur',
         'shear', 'scale',  'rotate',  'brightness',  'translate',
         'stripe', 'fog','spatter','dotted_line', 'zigzag',
         'canny_edges']


N_MINI_PER_CORRUPTION = 1000
    
@torch.no_grad()
def evaluate_cnn_on_mnistc_mini(corruption, cnn, max_batch_num=None):
    # get corruption batch information
    corruption_id = int(CORRUPTION_TYPES.index(corruption))
    num_batch_required = int(N_MINI_PER_CORRUPTION/BATCHSIZE) # if batchsize 100; 10 batches are requried
    
    # load dataloader and iterator
    dataloader = fetch_dataloader('mnist_c_mini', DATA_DIR, DEVICE, BATCHSIZE, train=False)    
    diter = iter(dataloader)
    
    # save output
    x_all, y_all, pred_all, acc_all, class_prob_all = [],[],[], [],[]
    cnn.eval() 

    # get input and gt
    for i in range(corruption_id*num_batch_required): #id =0, 0 iteration; id=1, 10 iteration
        x, y = next(diter)
    

    for i in range(0, num_batch_required):
        x, y = next(diter)
        gtx = None
        
        if max_batch_num:
            if i == max_batch_num:
                break
        
        data, target = x.to(DEVICE),  y.to(DEVICE)
        target = target.argmax(dim=1, keepdim=True)
        output = cnn(data)
        #                 test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        acc = pred.eq(target.view_as(pred))
        
        x_all.append(data)
        y_all.append(target.flatten())
        pred_all.append(pred.flatten())
        acc_all.append(acc.flatten().float())
        class_prob_all.append(output)
    

    # concat and add to outputs dictionary
    x_all = torch.cat(x_all, dim=0)
    y_all = torch.cat(y_all, dim=0)
    pred_all = torch.cat(pred_all, dim=0)
    acc_all = torch.cat(acc_all, dim=0)
    class_prob_all = torch.cat(class_prob_all, dim=0)

    return x_all, y_all, class_prob_all, pred_all, acc_all



@torch.no_grad()
def evaluate_cnn_on_mnistc_original(corruption, cnn):
    path_images = os.path.join(PATH_MNISTC, corruption, 'test_images.npy')
    path_labels = os.path.join(PATH_MNISTC, corruption, 'test_labels.npy')

    # convert to torch
    images = np.load(path_images)
    labels = np.load(path_labels)
    transform_tohot = T.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
    images_tensorized = torch.stack([T.ToTensor()(im) for im in images])
    labels_tensorized = torch.stack([transform_tohot(label) for label in labels])
    # print(images_tensorized.shape) #torch.Size([10000, 1, 28, 28])
    # print(labels_tensorized.shape) #torch.Size([10000, 10])

    # create dataloader
    kwargs = {'num_workers': 1, 'pin_memory': True} if DEVICE == 'cuda' else {}
    dataset = TensorDataset(images_tensorized, labels_tensorized)
    dataloader = DataLoader(dataset, batch_size=BATCHSIZE, shuffle=False, drop_last=False, **kwargs)

    # save output
    x_all, y_all, pred_all, acc_all, class_prob_all = [],[],[], [],[]
    cnn.eval() 

    # get input and gt

    for data in dataloader:
        x, y = data
        gtx = None
        
        data, target = x.to(DEVICE),  y.to(DEVICE)
        target = target.argmax(dim=1, keepdim=True)
        output = cnn(data)
        #                 test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        acc = pred.eq(target.view_as(pred))
        
        x_all.append(data)
        y_all.append(target.flatten())
        pred_all.append(pred.flatten())
        acc_all.append(acc.flatten().float())
        class_prob_all.append(output)
    

    # concat and add to outputs dictionary
    x_all = torch.cat(x_all, dim=0)
    y_all = torch.cat(y_all, dim=0)
    pred_all = torch.cat(pred_all, dim=0)
    acc_all = torch.cat(acc_all, dim=0)
    class_prob_all = torch.cat(class_prob_all, dim=0)

    return x_all, y_all, class_prob_all, pred_all, acc_all

# Model evaluation on MNIST-C

In [9]:
# task ='mnist_c_mini'
task ='mnist_c_original'

modellist = [
'./models/cnn/run1_epoch50.pt',
'./models/cnn/run2_epoch50.pt',
'./models/cnn/run3_epoch50.pt',
'./models/cnn/run4_epoch50.pt',
'./models/cnn/run5_epoch50.pt',
]

df = pd.DataFrame()
df['corruption'] = CORRUPTION_TYPES

for i, load_model_path in enumerate(modellist):
    print(f'start analysis on {load_model_path}')
    cnn = Net().to(DEVICE)
    cnn.load_state_dict(torch.load(load_model_path))
    cnn.eval()
    accs = []
    for corruption in CORRUPTION_TYPES:

        if task == 'mnist_c_mini':
            data_cnn, target_cnn, logsoft_cnn, pred_cnn, acc_cnn \
            = evaluate_cnn_on_mnistc_mini(corruption, cnn, max_batch_num=None)
#             print(f'==> corruption type: {corruption}, this batch acc: {acc_cnn.float().mean().item()}')
        elif task =='mnist_c_original':
#             print("original is used")
            data_cnn, target_cnn, logsoft_cnn, pred_cnn, acc_cnn \
            =  evaluate_cnn_on_mnistc_original(corruption, cnn)
#             print(f'==> corruption type: {corruption}, this batch acc: {acc_cnn.float().mean().item()}')

        accs.append(acc_cnn.float().mean().item())

    df[load_model_path]=accs

df.index = np.arange(1, len(df)+1)
df.loc['AVERAGE'] = df.mean()

df

start analysis on ./models/cnn/run1_epoch50.pt
start analysis on ./models/cnn/run2_epoch50.pt
start analysis on ./models/cnn/run3_epoch50.pt
start analysis on ./models/cnn/run4_epoch50.pt
start analysis on ./models/cnn/run5_epoch50.pt


/tmp/ipykernel_224080/3810698567.py:38: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.loc['AVERAGE'] = df.mean()


,corruption,./models/cnn/run1_epoch50.pt,./models/cnn/run2_epoch50.pt,./models/cnn/run3_epoch50.pt,./models/cnn/run4_epoch50.pt,./models/cnn/run5_epoch50.pt
1,identity,0.991300,0.991500,0.991100,0.990600,0.990500
2,shot_noise,0.968900,0.971300,0.973400,0.972600,0.974500
3,impulse_noise,0.903800,0.894400,0.922300,0.903500,0.893900
4,glass_blur,0.878800,0.889300,0.925200,0.869500,0.902600
5,motion_blur,0.891800,0.923200,0.937500,0.915000,0.927700
6,shear,0.973300,0.973600,0.973500,0.971900,0.974700
7,scale,0.937000,0.936300,0.936100,0.945300,0.936200
8,rotate,0.918900,0.919900,0.918000,0.915100,0.921000
9,brightness,0.867000,0.750200,0.883700,0.857100,0.809400
10,translate,0.475200,0.484200,0.472100,0.473400,0.484600


In [10]:
# save to csv
path_df = 'model-results-cnn.csv'
if os.path.isfile(path_df):
    print(f'test done! file {path_df} already exists, df is not saved')
else: 
    df.to_csv(path_df, index=False)
    print(f'test done! df is saved to csv as {path_df}')

test done! df is saved to csv as model-results-cnn.csv


# Model evaluation on each batch

In [40]:
task ='mnist_c_mini'
test_dataloader = fetch_dataloader(task, DATA_DIR, DEVICE, BATCHSIZE, train=False)

load_model_path = './results/mnist/cnn3/epoch50.pt'

model = Net().to(DEVICE)
model.load_state_dict(torch.load(load_model_path))
model.eval()


batchnum=3*int(1000/BATCHSIZE)-1

accs_mini = []
pred_mini =[]
target_mini =[]

diter = iter(test_dataloader)
for i in range(batchnum):
    x, y = next(diter)

with torch.no_grad():            
    data, target = x.to(DEVICE),  y.to(DEVICE)
    target = target.argmax(dim=1, keepdim=True)
    output = model(data)
#                 test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
    pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct = pred.eq(target.view_as(pred)).sum().item()
    acc = correct /BATCHSIZE
accs_mini.append(acc)
pred_mini.append(pred)
target_mini.append(target)

print(np.mean(accs_mini))

predictions = torch.cat(pred_mini, dim=0).flatten()
targets =  torch.cat(target_mini, dim=0).flatten()

## print predictions and acc
for i in range(len(predictions)):
    if predictions[i] != targets[i]:
        sign= "***"
    else: sign=None
    print(f'trial: {i}, prediction: {predictions[i]}, {sign}')

0.91
